In [1]:
sealed trait Animal
object Animal {
  final case class Dog(name: String) extends Animal
  final case class Cat(name: String) extends Animal
}
trait Vet[-T] {
  def heal(animal: T): Boolean
}
val myDog = Animal.Dog("Buddy")
val felix: Animal = Animal.Cat("Felix")
val myVet: Vet[Animal.Dog] = new Vet[Animal.Dog] {
    def heal(animal: Animal.Dog) = {
       true
    }
}

defined trait Animal
defined object Animal
defined trait Vet
myDog: Animal.Dog = Dog("Buddy")
felix: Animal = Cat(Felix)
myVet: Vet[Animal.Dog] = $sess.cmd0Wrapper$Helper$$anon$1@703ba5

# Functions variance

Functions are contravariant in their arguments, and covariant in their results. Why? If a functions accepts one type, then it can accept any of it's subtypes. For example, if I know how to name an animal, I know how to name cats in particular:

In [6]:
val f1 : Animal     => String = _.toString
val f2 : Animal.Cat => String = f1
val f3 : Animal.Dog => String = f1

f1: Animal => String = <function1>
f2: Animal.Cat => String = <function1>

On the other hand, if a function returns an specific type, then that result can also be viewed as any of it's supertypes. For example, if I know how to clone a Cat then I know how to clone _some_ animal:

In [11]:
val f1 : Unit => Animal.Cat = _ => Animal.Cat("minino")
val f2 : Unit => Animal     = f1

// Same thing happens with other subtypes
val f3 : Unit => Animal.Dog = _ => Animal.Dog("pulgas")
val f4 : Unit => Animal     = f3

f1: Unit => Animal.Cat = <function1>
f2: Unit => Animal = <function1>
f3: Unit => Animal.Dog = <function1>
f4: Unit => Animal = <function1>

# Variance restrictions in functions

Because of the covariance restrictions over functions you cannot put a covariant type in as an argument:

In [2]:
trait Vet[+T] {
  def heal(animal: T): Boolean
}

cmd2.sc:2: covariant type T occurs in contravariant position in type T of value animal
  def heal(animal: T): Boolean
           ^

: 

Similarly, you cannot put a contravariant type as a result:

In [ ]:
trait Lab[-T] {
  def clone(): T
}

Why these restrictions?

First, a mental exercise for covariance:

In [3]:
trait Vet[+T] {
   def heal(animal: Any): Boolean // IMAGINE it's not Any but T
}

val notReallyAGeneralVet: Vet[Animal] = new Vet[Animal.Cat] { // This works because of covariance: Cat <: Animal => Vet[Cat] <: Vet[Animal]
     def heal(animal: Any): Boolean = { // IMAGINE it's not Any but Cat
       animal match {
         case Animal.Cat(_) => 
            true
         case other =>
            throw new Exception(s"Oh not, got an unexpected ${other.getClass}")
       }
     }
} 

val dog: Animal.Dog = Animal.Dog("Pulgas")

notReallyAGeneralVet.heal(dog) // this invocation, if it had compiled with a generic type T, would be unsafe.

: 

Allowing covariance in arguments makes a method **forget** the specific type it works with, that's why it's not permitted. It allows us to "lie": we can turn a method that only receives a specific subset of values into one that receives a more general set of values.

In the example above, covariance would have made a function that only works accepting `Cats` be allowed to be invoked _an_ `Animal`, in particular one that's not a `Cat`, like a `Dog`. This is an error that the variance annotations can help catch at compile time.

Now, a mental exercise with contravariance:

In [4]:
trait Lab[-T] {
  def cloneAnimal(): Any // IMAGINE it's not Any but T
}

val notReallyACatCloner: Lab[Animal.Cat] = new Lab[Animal] { // This works because of contravariance: Cat <: Animal => Lab[Animal] <: Lab[Cat]
  def cloneAnimal(): Any = // IMAGINE it's not Any but Animal
     Animal.Dog("pulgas")
}

val cat = notReallyACatCloner.cloneAnimal().asInstanceOf[Animal.Cat] // this invocation, if cloneAnimal returned type T, should 

: 